<a href="https://colab.research.google.com/github/Madhusudhanbandi/EVA/blob/master/EVA4_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) 
        self.bnorm1 = nn.BatchNorm2d(8)
        self.drop1 = nn.Dropout2d(0.2)
        self.conv2 = nn.Conv2d(8, 12, 3, padding=1)
        self.bnorm2 = nn.BatchNorm2d(12)
        self.drop1 = nn.Dropout2d(0.2)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(12, 16, 3, padding=1)
        self.bnorm3 = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 20, 3, padding=1)
        self.bnorm4 = nn.BatchNorm2d(20)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(20, 24, 3)
        self.bnorm5 = nn.BatchNorm2d(24)
        self.conv6 = nn.Conv2d(24, 32, 3)
        self.conv7 = nn.Conv2d(32, 10, 3)
        # self.conv8 = nn.AvgPool2d(2,1)

    def forward(self, x):
        x = self.pool1(self.bnorm2(F.relu(self.drop1(self.conv2(self.bnorm1(F.relu(self.drop1(self.conv1(x)))))))))
        x = self.pool2(self.bnorm4(F.relu(self.conv4(self.bnorm3(F.relu(self.drop1(self.conv3(x))))))))
        x = F.relu(self.conv6(self.bnorm5(F.relu(self.conv5(x)))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [60]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
         Dropout2d-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
            Conv2d-4           [-1, 12, 28, 28]             876
         Dropout2d-5           [-1, 12, 28, 28]               0
       BatchNorm2d-6           [-1, 12, 28, 28]              24
         MaxPool2d-7           [-1, 12, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           1,744
         Dropout2d-9           [-1, 16, 14, 14]               0
      BatchNorm2d-10           [-1, 16, 14, 14]              32
           Conv2d-11           [-1, 20, 14, 14]           2,900
      BatchNorm2d-12           [-1, 20, 14, 14]              40
        MaxPool2d-13             [-1, 20, 7, 7]               0
           Conv2d-14             [-1, 2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


# torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() #clears old gradients from the last step (otherwise you’d just accumulate the gradients from all loss.backward() calls).
        output = model(data) # outputof the model means which class
        loss = F.nll_loss(output, target) #loss calculation output(predicted) - target(actual)
        loss.backward()  #  computes the derivative of the loss w.r.t. the parameters (or anything requiring gradients) using backpropagation.
        optimizer.step() # causes the optimizer to take a step based on the gradients of the parameters.
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [65]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1,20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.08237438648939133 batch_id=0:   0%|          | 1/469 [00:00<01:18,  5.99it/s]


Test set: Average loss: 0.0575, Accuracy: 9804/10000 (98%)



loss=0.040244899690151215 batch_id=0:   0%|          | 1/469 [00:00<01:15,  6.17it/s]


Test set: Average loss: 0.0363, Accuracy: 9878/10000 (99%)



loss=0.08382202684879303 batch_id=0:   0%|          | 1/469 [00:00<01:14,  6.30it/s]

...


Test set: Average loss: 0.0218, Accuracy: 9932/10000 (99%)



loss=0.009424526244401932 batch_id=0:   0%|          | 1/469 [00:00<01:16,  6.14it/s]


Test set: Average loss: 0.0230, Accuracy: 9929/10000 (99%)



loss=0.009703089483082294 batch_id=468: 100%|██████████| 469/469 [01:16<00:00,  6.17it/s]



Test set: Average loss: 0.0195, Accuracy: 9934/10000 (99%)

